In [1]:
import torch, csv, emoji
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from transformers import AutoModel, AutoTokenizer 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer

from NN import read_csv, emojiDataset, toEmoji

In [2]:
a = read_csv('490A final project data - Kai Processing.csv')
print(a[0][-1], a[1][-1])

That's your dream? Aim higher babe ['💭', '🎯']


In [3]:
dataset = emojiDataset()

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
data_loader = DataLoader(dataset=dataset, batch_size=10, shuffle=True)
data_iter = iter(data_loader)
sentences, emojis, labels_classes = dataset.get_others()
# next_data = data_iter.next()
# features, labels = next_data
# print(features, labels)

toemoji = toEmoji()
learn_rate = optim.Adam(toemoji.parameters(), lr=0.01)
loss_func = nn.MSELoss()
epochs = 5

In [5]:
for i in range(epochs):
    for data in data_loader:
        features, labels = data
        toemoji.zero_grad()
        results = toemoji(features.float())
        # print(result)
        loss = loss_func(results, labels)
        # for result, label in zip(results, labels):
        #     loss += F.nll_loss(result, label) 
        loss.backward()
        learn_rate.step()
    print(loss)

tensor(0.0137, grad_fn=<MseLossBackward0>)
tensor(0.0086, grad_fn=<MseLossBackward0>)
tensor(0.0092, grad_fn=<MseLossBackward0>)
tensor(0.0101, grad_fn=<MseLossBackward0>)
tensor(0.0102, grad_fn=<MseLossBackward0>)


In [6]:
K = 10

with torch.no_grad():
    for data in data_loader:
        features, labels = data
        results = toemoji(features.float())
        for index, result in enumerate(results):
                # temp = []
                # for elem in result:
                #     if elem > 0: temp.append(1)
                #     else: temp.append(0)
                # # print(result)
                original_str, result_str = '', ''

                top_k = (-result).argsort()[:K]
                for top_index in top_k:
                    result_str += labels_classes[top_index]

                original_str = ''.join(emojis[index])
                
                print(f'Sentence: {sentences[index]}\nOriginal labels: {original_str}, output labels: {result_str}\n')

Sentence: eat then, idiot.
Original labels: 🍽🤦, output labels: 👨❌👉🗣❤🤔👍👩📅🤙

Sentence: Loving you and having you  is  the most successful achievement in life ♥️✨
Original labels: ♥👤, output labels: 👨❌👍👉❤🤔🗣👩😭👀

Sentence: Imagine not being arab 💀💀😭
Original labels: ❌👳, output labels: 👨❌👍👉❤🤔🗣👩😭👀

Sentence: I’m going to a concert tonight and I’m so nervous🤩
Original labels: 🎤🌃, output labels: 👨❌👉👍❤🤔🗣👩😭📅

Sentence: gatekeeping painting…..
Original labels: 👀🖼, output labels: 👨❌👉👍❤🤔🗣👩😭📅

Sentence: Not if I see you in water
Original labels: ❌🌊, output labels: 👨❌👍👉❤🤔🗣👩😭👀

Sentence: yowdi i smell small chismis.
Original labels: 👨👃, output labels: 👨❌👉👍❤🤔🗣👩😭📅

Sentence: A lot of them niggas🤧
Original labels: 👨🏿🤧, output labels: 👨❌👉🗣❤👍🤔👩📅😭

Sentence: jeez
Original labels: 😲, output labels: 👨❌👍👉❤🤔🗣👩😭📅

Sentence: This panther needs Calcium Chloride!
Original labels: 😷💊, output labels: 👨❌👍❤👉🤔🗣👩😭👀

Sentence: eat then, idiot.
Original labels: 🍽🤦, output labels: 👨❌👉👍❤🗣🤔👩😭📅

Sentence: Loving you and having 